<a href="https://colab.research.google.com/github/Arjavjain100/TOS-Summarization/blob/main/Pegasus_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install sentencepiece
!git clone "https://github.com/Arjavjain100/TOS-Summarization.git"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 8.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.6 MB/s eta 0:00:00
Cloning into 'TOS-Summarization'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 83 (delta 40), reused 24 (delta 11), pack-reused 0
Unpacking objects: 100% (83/83), 713.95 KiB | 4.49 MiB/s, done.


In [73]:
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [74]:
import torch
from transformers import pipeline,AutoModelForSeq2SeqLM, AutoTokenizer, PreTrainedTokenizer, PegasusTokenizer, PegasusForConditionalGeneration, PretrainedConfig
import pandas as pd
from rouge import Rouge

In [4]:
df = pd.read_csv("/content/TOS-Summarization/Dataset/all_v1_transpose.csv")
df = df[['original_text','reference_summary']]
df.rename(columns = {'original_text':'source', 'reference_summary':'target'}, inplace = True)

In [5]:
df.head()

,source,target
0,welcome to the pokémon go video game services ...,hi.
1,by using our services you are agreeing to thes...,by playing this game you agree to these terms....
2,if you want to use certain features of the ser...,you have to use google pokemon trainer club or...
3,during game play please be aware of your surro...,don t die or hurt others and if you do it s no...
4,subject to your compliance with these terms ni...,don t copy modify resell distribute or reverse...


In [102]:
X = df['source']
y = df['target']

In [7]:
X[42]

'account termination policy youtube will terminate a user s access to the service if under appropriate circumstances the user is determined to be a repeat infringer. youtube reserves the right to decide whether content violates these terms of service for reasons other than copyright infringement such as but not limited to pornography obscenity or excessive length. youtube may at any time without prior notice and in its sole discretion remove such content and or terminate a user s account for submitting such material in violation of these terms of service.'

In [42]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
config = PretrainedConfig.from_json_file('/content/gdrive/MyDrive/Minor Project 2/ouput_model/config.json')

In [45]:
tokenizer = PegasusTokenizer.from_pretrained('nsi319/legal-pegasus', config=config)

In [44]:
model = PegasusForConditionalGeneration.from_pretrained("/content/gdrive/MyDrive/Minor Project 2/ouput_model", config = config).to(device)

In [62]:
def summarize(text):
  input_tokenized = tokenizer.encode(text, return_tensors='pt',max_length=1024,truncation=True).to(device)
  summary_ids = model.generate(input_tokenized,
                                  num_beams=9,
                                  no_repeat_ngram_size=3,
                                  length_penalty=2.0,
                                  min_length=50,
                                  max_length=150,
                                  early_stopping=True)
  summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]

  return summary

In [66]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)     

In [67]:
y_pred = X_test.apply(lambda x: summarize(x))

In [68]:
type(y_pred)

pandas.core.series.Series

In [71]:
y_pred.iloc[0]

'the service may use industry wide tracking tracking methods e g cookies web beacons etc. which store certain information and may allow third parties to collect certain data for the purpose of serving advertisements through the use of the novelty services without limitation to the user’s consent.'

In [72]:
y_test.iloc[0]

'this service allows tracking via third party cookies for purposes including targeted advertising.'

In [96]:
summary = pd.concat([y_test.to_frame(name="reference_summary"), y_pred.to_frame(name="generated_summary")], axis=1)

In [118]:
summary.head()

,reference_summary,generated_summary,rouge
438,this service allows tracking via third party c...,the service may use industry wide tracking tra...,"{'rouge-1': {'r': 0.38461538461538464, 'p': 0...."
415,the service may keep a secure anonymized recor...,this service may delete your personal data if ...,"{'rouge-1': {'r': 0.3888888888888889, 'p': 0.1..."
301,you are responsible for maintaining the securi...,you are responsible for the content that you p...,"{'rouge-1': {'r': 0.5384615384615384, 'p': 0.2..."
409,this service is only available to users of a c...,if you are under 18 you must have a parent gu...,"{'rouge-1': {'r': 0.36363636363636365, 'p': 0...."
141,this service can share your personal informati...,this service may transfer your personal inform...,"{'rouge-1': {'r': 0.9, 'p': 0.2647058823529412..."


In [119]:
rouge = Rouge()

In [120]:
rouge.get_scores(summary['generated_summary'], summary['reference_summary'],avg=True)

{'rouge-1': {'r': 0.4888778718360985,
  'p': 0.1836553040220889,
  'f': 0.2520288613757483},
 'rouge-2': {'r': 0.19016277934407527,
  'p': 0.05461454075588693,
  'f': 0.07851277090682468},
 'rouge-l': {'r': 0.4404436056901954,
  'p': 0.16249748049024254,
  'f': 0.223715507530049}}